## Models

### LLMResponse

In [1]:
from sdk import LLMResponse, ToolCall

In [2]:
# ToolCall example
ToolCall(
    name="read_file",
    arguments={"file": "file_name.txt"}
    )

ToolCall(name='read_file', tool_call_id=None, arguments={'file': 'file_name.txt'})

In [3]:
# - text : str
# - toolcall : Optional[List[ToolCall]]
# -------------------------------------
# LLM returns only text
LLMResponse(
    text="Hello, World!"
    )

LLMResponse(text='Hello, World!', tool_calls=None)

In [4]:
# LLM returns one or more tool calls
calls = [
    ToolCall(
        name="read_file",
        arguments=
            {"file": "file_name.txt"}
        ),
    ToolCall(
        name="write_file",
        arguments=
            {"file": "output.txt",
             "content": "Hello, World!"}
        )
    ]

LLMResponse(
    tool_calls=calls
    )

LLMResponse(text=None, tool_calls=[ToolCall(name='read_file', tool_call_id=None, arguments={'file': 'file_name.txt'}), ToolCall(name='write_file', tool_call_id=None, arguments={'file': 'output.txt', 'content': 'Hello, World!'})])

In [5]:
# LLM returns text and tool calls
LLMResponse(
    text="Here are the results:",
    tool_calls=calls
    )

LLMResponse(text='Here are the results:', tool_calls=[ToolCall(name='read_file', tool_call_id=None, arguments={'file': 'file_name.txt'}), ToolCall(name='write_file', tool_call_id=None, arguments={'file': 'output.txt', 'content': 'Hello, World!'})])

### Messages

In [6]:
from sdk import (ToolResult, Message, UserMessage, AIMessage, SystemMessage,ToolMessage)

In [7]:
ToolResult(success=True)

ToolResult(success=True, result=None, error=None)

In [8]:
ToolResult(success=True,
           result="File read successfully.")

ToolResult(success=True, result='File read successfully.', error=None)

In [9]:
tr = ToolResult(success=False,
                error="File not found.")

tr

ToolResult(success=False, result=None, error='File not found.')

In [10]:
UserMessage(content="Hello, how are you?")

UserMessage(role='user', content='Hello, how are you?')

In [11]:
SystemMessage(content="You are a helpful assistant.")

SystemMessage(role='system', content='You are a helpful assistant.')

In [12]:
AIMessage(content="I'm doing well, thank you!")

AIMessage(role='assistant', content="I'm doing well, thank you!")

In [13]:
ToolMessage(content=tr)

ToolMessage(role='tool', content=ToolResult(success=False, result=None, error='File not found.'), tool_call_id=None)

In [14]:
Message.__args__

(sdk.models.models.UserMessage,
 sdk.models.models.AIMessage,
 sdk.models.models.SystemMessage,
 sdk.models.models.ToolMessage)

## Memory

In [15]:
from sdk import Memory

In [16]:
history = Memory()
history.get_messages()

[]

In [17]:
history.add_system_message("You are a helpful assistant.")
history.get_messages()

[SystemMessage(role='system', content='You are a helpful assistant.')]

In [18]:
history.add_user_message("What is the weather like today?")
history.get_messages()

[SystemMessage(role='system', content='You are a helpful assistant.'),
 UserMessage(role='user', content='What is the weather like today?')]

In [19]:
response = LLMResponse(
    tool_calls=[
        ToolCall(
            name="get_weather",
            arguments={"location": "San Francisco"},
            tool_call_id="tc_12345"
        )
    ]
    )
history.add_assistant_message(response)
history.get_messages()

[SystemMessage(role='system', content='You are a helpful assistant.'),
 UserMessage(role='user', content='What is the weather like today?'),
 AIMessage(role='assistant', content=LLMResponse(text=None, tool_calls=[ToolCall(name='get_weather', tool_call_id='tc_12345', arguments={'location': 'San Francisco'})]))]

In [20]:
tr2 = ToolResult(success=True,
                    result="The weather in San Francisco is sunny with a high of 75°F.")

tc_id = response.tool_calls[0].tool_call_id
history.add_tool_message(tr2, tool_call_id=tc_id)
history.get_messages()

[SystemMessage(role='system', content='You are a helpful assistant.'),
 UserMessage(role='user', content='What is the weather like today?'),
 AIMessage(role='assistant', content=LLMResponse(text=None, tool_calls=[ToolCall(name='get_weather', tool_call_id='tc_12345', arguments={'location': 'San Francisco'})])),
 ToolMessage(role='tool', content=ToolResult(success=True, result='The weather in San Francisco is sunny with a high of 75°F.', error=None), tool_call_id='tc_12345')]

In [21]:
response = LLMResponse(
    text="The weather in San Francisco is sunny with a high of 75°F."
    )
history.add_assistant_message(response)
history.get_messages()

[SystemMessage(role='system', content='You are a helpful assistant.'),
 UserMessage(role='user', content='What is the weather like today?'),
 AIMessage(role='assistant', content=LLMResponse(text=None, tool_calls=[ToolCall(name='get_weather', tool_call_id='tc_12345', arguments={'location': 'San Francisco'})])),
 ToolMessage(role='tool', content=ToolResult(success=True, result='The weather in San Francisco is sunny with a high of 75°F.', error=None), tool_call_id='tc_12345'),
 AIMessage(role='assistant', content=LLMResponse(text='The weather in San Francisco is sunny with a high of 75°F.', tool_calls=None))]

## Tools

In [22]:
from sdk import ReadFileTool, WriteFileTool

In [23]:
# Suppose that the worskspace where the tools can access is at src\workspace -> Default
r = ReadFileTool()

In [24]:
r.run(file_path="nonexisting.txt")

ToolResult(success=False, result=None, error="File 'nonexisting.txt' not found.")

In [25]:
r.run(file_path="example.txt")

ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated cylinder!\n8. Why do chemists like nitrates so much? Because they're cheaper than day rates!\n9. How did the chemist survive the explosion? He had a lot of good reactions!\n10. Why did the chemist bring a ladder to the bar? Because he heard the drinks were on the house!", error=None)

In [26]:
read_file_tool = ReadFileTool()
write_file_tool = WriteFileTool()

tools = [read_file_tool, write_file_tool]

# Tools are now used directly with Agent, no registry needed
print(f"Available tools: {[tool.name for tool in tools]}")

Available tools: ['read_file', 'write_file']


In [27]:
response = LLMResponse(
            tool_calls=[
                ToolCall(
                    name="read_file",
                    arguments={"file_path": "example.txt"}
                ),
                ToolCall(
                    name="write_file",
                    arguments={
                        "file_path": "output.txt",
                        "content": "This is a test."
                    }
                )
            ]
            )

# Execute tools directly
result = []
for call in response.tool_calls:
    # Find the tool by name
    tool = next((t for t in tools if t.name == call.name), None)
    if tool:
        result.append(tool.run(**call.arguments))
    else:
        result.append(ToolResult(success=False, error=f"Tool '{call.name}' not found"))

result

[ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated cylinder!\n8. Why do chemists like nitrates so much? Because they're cheaper than day rates!\n9. How did the chemist survive the explosion? He had a lot of good reactions!\n10. Why did the chemist bring a ladder to the bar? Because he heard the drinks were on the house!", error=None),
 ToolResult(success=True, result="File written to 'output.txt'.", error=None)]

## LLM

In [28]:
from sdk import Factory

In [29]:
llm = Factory().create()

In [30]:
test_history = Memory()
test_history.add_system_message("You are a helpful file management assistant.")
test_history.add_user_message("Can you read the example.txt file?")

test_history.get_messages()

[SystemMessage(role='system', content='You are a helpful file management assistant.'),
 UserMessage(role='user', content='Can you read the example.txt file?')]

In [31]:
# Get tool schemas directly from the tools
tool_schema = [tool.get_schema() for tool in tools]

In [32]:
response = llm.generate_text(
    messages=test_history.get_messages(),
    tools=tool_schema
)
response

LLMResponse(text=None, tool_calls=[ToolCall(name='read_file', tool_call_id='call_KUu84ORTQBLk2KM7G6tlIwOU', arguments={'file_path': 'example.txt'})])

In [33]:
results = []
for tool_call in response.tool_calls:
    # Find and execute the tool
    tool = next((t for t in tools if t.name == tool_call.name), None)
    if tool:
        tool_result = tool.run(**tool_call.arguments)
        results.append(tool_result)
    else:
        results.append(ToolResult(success=False, error=f"Tool '{tool_call.name}' not found"))

print(results)

[ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated cylinder!\n8. Why do chemists like nitrates so much? Because they're cheaper than day rates!\n9. How did the chemist survive the explosion? He had a lot of good reactions!\n10. Why did the chemist bring a ladder to the bar? Because he heard the drinks were on the house!", error=None)]


In [34]:
tool_call_id = response.tool_calls[0].tool_call_id

test_history.add_tool_message(results[0], tool_call_id=tool_call_id)
test_history.get_messages()

[SystemMessage(role='system', content='You are a helpful file management assistant.'),
 UserMessage(role='user', content='Can you read the example.txt file?'),
 ToolMessage(role='tool', content=ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated cylinder!\n8. Why do chemists like nitrates so much? Because they're cheaper than day rates!\n9. How did the chemist survive the explosion? He had a lot of good reactions!\n10. Why did the 

In [35]:
test_history.add_user_message("Please write the following content to example.txt: 'Hello, from the AI.'")
test_history.get_messages()

[SystemMessage(role='system', content='You are a helpful file management assistant.'),
 UserMessage(role='user', content='Can you read the example.txt file?'),
 ToolMessage(role='tool', content=ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated cylinder!\n8. Why do chemists like nitrates so much? Because they're cheaper than day rates!\n9. How did the chemist survive the explosion? He had a lot of good reactions!\n10. Why did the 

In [36]:
llm = Factory().create()
read_tool = ReadFileTool()
write_tool = WriteFileTool()
tools_final = [read_tool, write_tool]
tool_schema = [tool.get_schema() for tool in tools_final]

test_history_final = Memory()
test_history_final.add_system_message("You are a helpful file management assistant.")
test_history_final.add_user_message("Can you read the example.txt file?")

response = llm.generate_text(
    messages=test_history_final.get_messages(),
    tools=tool_schema
)
response

LLMResponse(text=None, tool_calls=[ToolCall(name='read_file', tool_call_id='call_CiwNoqBudpWw1VtLSJIulJ7S', arguments={'file_path': 'example.txt'})])

In [37]:
test_history_final.add_assistant_message(response)
# Execute tool
tool = next((t for t in tools_final if t.name == response.tool_calls[0].name), None)
result = tool.run(**response.tool_calls[0].arguments) if tool else ToolResult(success=False, error="Tool not found")
print(f"✓ Tool result: {result}")

# Add tool message
test_history_final.add_tool_message(result, tool_call_id=response.tool_calls[0].tool_call_id)
test_history_final.get_messages()

✓ Tool result: success=True result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated cylinder!\n8. Why do chemists like nitrates so much? Because they're cheaper than day rates!\n9. How did the chemist survive the explosion? He had a lot of good reactions!\n10. Why did the chemist bring a ladder to the bar? Because he heard the drinks were on the house!" error=None


[SystemMessage(role='system', content='You are a helpful file management assistant.'),
 UserMessage(role='user', content='Can you read the example.txt file?'),
 AIMessage(role='assistant', content=LLMResponse(text=None, tool_calls=[ToolCall(name='read_file', tool_call_id='call_CiwNoqBudpWw1VtLSJIulJ7S', arguments={'file_path': 'example.txt'})])),
 ToolMessage(role='tool', content=ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated 

In [38]:
response = llm.generate_text(
    messages=test_history_final.get_messages(),
    tools=tool_schema
)

In [39]:
test_history_final.add_assistant_message(response)
test_history_final.get_messages()

[SystemMessage(role='system', content='You are a helpful file management assistant.'),
 UserMessage(role='user', content='Can you read the example.txt file?'),
 AIMessage(role='assistant', content=LLMResponse(text=None, tool_calls=[ToolCall(name='read_file', tool_call_id='call_CiwNoqBudpWw1VtLSJIulJ7S', arguments={'file_path': 'example.txt'})])),
 ToolMessage(role='tool', content=ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated 

In [40]:
# Add next user message
test_history_final.add_user_message("Please write the following content to example.txt: 'Hello, from the AI.'")
response = llm.generate_text(
    messages=test_history_final.get_messages(),
    tools=tool_schema
)
test_history_final.add_assistant_message(response)
test_history_final.get_messages()


[SystemMessage(role='system', content='You are a helpful file management assistant.'),
 UserMessage(role='user', content='Can you read the example.txt file?'),
 AIMessage(role='assistant', content=LLMResponse(text=None, tool_calls=[ToolCall(name='read_file', tool_call_id='call_CiwNoqBudpWw1VtLSJIulJ7S', arguments={'file_path': 'example.txt'})])),
 ToolMessage(role='tool', content=ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated 

In [41]:
# Execute tool
tool = next((t for t in tools_final if t.name == response.tool_calls[0].name), None)
result = tool.run(**response.tool_calls[0].arguments) if tool else ToolResult(success=False, error="Tool not found")
print(f"✓ Tool result: {result}")

# Add tool message
test_history_final.add_tool_message(result, tool_call_id=response.tool_calls[0].tool_call_id)
test_history_final.get_messages()

✓ Tool result: success=True result="File written to 'example.txt'." error=None


[SystemMessage(role='system', content='You are a helpful file management assistant.'),
 UserMessage(role='user', content='Can you read the example.txt file?'),
 AIMessage(role='assistant', content=LLMResponse(text=None, tool_calls=[ToolCall(name='read_file', tool_call_id='call_CiwNoqBudpWw1VtLSJIulJ7S', arguments={'file_path': 'example.txt'})])),
 ToolMessage(role='tool', content=ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated 

In [42]:
response = llm.generate_text(
    messages=test_history_final.get_messages(),
    tools=tool_schema
)
test_history_final.add_assistant_message(response)
test_history_final.get_messages()

[SystemMessage(role='system', content='You are a helpful file management assistant.'),
 UserMessage(role='user', content='Can you read the example.txt file?'),
 AIMessage(role='assistant', content=LLMResponse(text=None, tool_calls=[ToolCall(name='read_file', tool_call_id='call_CiwNoqBudpWw1VtLSJIulJ7S', arguments={'file_path': 'example.txt'})])),
 ToolMessage(role='tool', content=ToolResult(success=True, result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated 

In [43]:
iter = test_history_final.get_messages().__iter__()
for i in iter:
    print(i.role)
    print(i.content)
    print("-----")
    print()

system
You are a helpful file management assistant.
-----

user
Can you read the example.txt file?
-----

assistant
text=None tool_calls=[ToolCall(name='read_file', tool_call_id='call_CiwNoqBudpWw1VtLSJIulJ7S', arguments={'file_path': 'example.txt'})]
-----

tool
success=True result="1. Why did the physicist break up with the biologist? There was no chemistry.\n2. Why can't you trust an atom? Because they make up everything!\n3. What did the scientist say when he found two isotopes of helium? HeHe!\n4. Why are chemists excellent for solving problems? They have all the solutions.\n5. What do you do with a sick chemist? If you can't helium, and you can't curium, then you might as well barium!\n\n6. Why did the chemist get kicked off the plane? Because he kept insisting that he had to carry his own solutions!\n7. What do you call an educated tube? A graduated cylinder!\n8. Why do chemists like nitrates so much? Because they're cheaper than day rates!\n9. How did the chemist survive the ex

## Not Related to sdk

In [44]:
def openai_tool_schema(tools):
    openai_tools = []
    for tool in tools:

        if hasattr(tool, 'get_schema'):
            schema = tool.get_schema()

        elif isinstance(tool, dict):
            schema = tool
        else:
            raise ValueError(f"Invalid tool type: {type(tool)}")

        openai_tools.append({
            "type": "function",
            "function": {
                "name": schema.get("name"),
                "description": schema.get("description"),
                "parameters": schema.get("parameters", {})
            }
            })
    return openai_tools

In [45]:
from openai import OpenAI
import os

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

agent_tools = [ReadFileTool(), WriteFileTool()]
tool_schema = [tool.get_schema() for tool in agent_tools]

msg = []
msg.append({"role": "system", "content": "You are a helpful file management assistant."})
msg.append({"role": "user", "content": "Can you read the example.txt file?"})

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=msg,
    tools=openai_tool_schema(tools=tool_schema),
    temperature=0,
)

In [46]:
message = response.choices[0].message
tool_calls_list = []

if message.tool_calls:
    for tc in message.tool_calls:
        for i in tc:
            print(i)

('id', 'call_NLB7r7BpuElSL7TlifSGE0pp')
('function', Function(arguments='{"file_path":"example.txt"}', name='read_file'))
('type', 'function')
